In [1]:
import pandas as pd
df = pd.read_csv('c:/data/pima/data.csv')
df

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,Diabetes_Pedigree_Function,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [2]:
df.columns

Index(['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin',
       'bmi', 'Diabetes_Pedigree_Function', 'age', 'outcome'],
      dtype='object')

In [3]:
X = df[['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin',
       'bmi', 'Diabetes_Pedigree_Function', 'age']]
y = df['outcome']

In [9]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
#refit='AUC' 추정값을 교차검증한 AUC 점수가 가장 좋은 매개변서
#make_scorer() 사용자정의 평가함수를 지정하는 함수
#
scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
    param_grid={'min_samples_spilt': range(2, 403, 10)},
    scoring=scoring, refit='AUC', return_train_score=True)

gs.fit(X, y)
results = gs.cv_results_
results

ValueError: Invalid parameter min_samples_spilt for estimator DecisionTreeClassifier(random_state=42). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
print(results['mean_test_AUC'])
print(results['mean_test_Accuray'])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

plt.figure(figsize=(13,13))
plt.xlabel('min_samples_split')
plt.ylabel('Score')

ax = plt.gca()
ax.set_xlim(0,402)
ax.set_ylim(0.73,1)

X_axis = np.array(results['param_min_samples_split'].data, dtype=float)
for scorer, color in zip(sorted(scoring), ['g', 'k']):
    for sample, style in (('train', '--'), ('test', '-')):
        sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
        ax.plot(X_axis, sample_score_mean, style, color=color, label='%s (%s)' % (scorer, sample))

best_index = np.argmax(results['mean_test_%s' % scorer])
best_score = results['mean_test_%s' % scorer][best_index]

ax.plot([X_axis[best_index],]*2,[0,best_score],
linestyle='-.', color=color, marker='o', ms=20)

In [ ]:
ax.annotate('0.2f' % best_score,
(X_axis[best_index], best_score + 0.005))

plt.legend()
plt.grid(False)
plt.show()